harika! **Credit Card Fraud (ULB)** ile SOM egzersizini adım adım, Jupyter’da direkt çalışacak şekilde veriyorum. Kod, veriyi yükleme → ölçekleme → (opsiyonel) PCA → **SOM eğitimi** → **U-Matrix görselleştirme** → **Fraud noktalarını haritada gösterme** → **SOM tabanlı anomaly skoru + ROC/PR** → **K-Means + Elbow** akışını kapsar.

> Not: Aşağıdaki kod, dosya adının `creditcard.csv` olduğunu varsayar (Kaggle’dan inince öyle geliyor). Eğer farklı klasördeyse PATH’i değiştir.

---

# 0) Kurulum (gerekirse)

```python
# (Gerekirse) MiniSom kurulumu
# !pip install minisom --quiet
```

---

# 1) Kütüphaneler ve uyarıları kısma

```python
import os, warnings, logging
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, classification_report

from minisom import MiniSom
from sklearn.cluster import KMeans
```

---

# 2) Veri yükleme + hızlı bakış

```python
# CSV dosyasını oku
df = pd.read_csv("creditcard.csv")

print(df.shape)
print(df.head(3))

# Sütunlar tipik olarak: ['Time','V1'..'V28','Amount','Class']
assert "Class" in df.columns, "Class etiketi bulunamadı"
y = df["Class"].values.astype(int)

# Özellik matrisi
X = df.drop(columns=["Class"])
```

**Neden?**

* `Class` (0=normal, 1=fraud) etiketi değerlendirme için kalsın; **SOM eğitimi etiketsizdir (unsupervised).**

---

# 3) Ölçekleme + (Opsiyonel) PCA

**Neden?** SOM öklid mesafesiyle çalışır; ölçek farklılıkları sonuçları bozar. PCA gürültüyü azaltıp eğitimi hızlandırabilir.

```python
# "Amount" ve "Time" ölçeklenir; V1..V28 zaten PCA-türevi (ama yine de standardization faydalıdır)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Opsiyonel PCA: %95 varyansı koru (dilersen kapatabilirsin)
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print("Orijinal boyut:", X_scaled.shape[1], "-> PCA boyutu:", X_pca.shape[1])
```

> İlk denemede **PCA açık** bırakmanı öneririm (eğitim hızlanır). İstersen daha sonra PCA’yı kapatıp sadece `X_scaled` ile dene.

---

# 4) Train/Test böl (etiketi sadece değerlendirme için tutuyoruz)

```python
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, stratify=y, random_state=42
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()
```

---

# 5) SOM parametreleri ve eğitim

**Neden bu değerler?** 20×20 grid orta boy; `sigma` komşuluk yayılımı, `learning_rate` başlangıç öğrenme hızı. İterasyon sayısını veri boyutuna göre artırıp azaltabilirsin.

```python
m, n = 20, 20                   # SOM ızgara boyutu (20x20)
input_len = X_train.shape[1]    # PCA sonrası özellik sayısı
sigma = 2.0
lr = 0.5

som = MiniSom(x=m, y=n, input_len=input_len, sigma=sigma, learning_rate=lr,
              neighborhood_function='gaussian', random_seed=42)

som.random_weights_init(X_train)
print("SOM init tamam.")

# Eğitim (iteration = yaklaşık 10 * num_samples iyi bir başlangıç)
iterations = 10 * X_train.shape[0]
som.train_random(X_train, iterations, verbose=True)
print("SOM eğitim bitti.")
```

> Daha hızlı deneme için `iterations = 5 * X_train.shape[0]` da yeterli olabilir. Stabil hale gelince artırırsın.

---

# 6) U-Matrix (distance map) görselleştirme

**Neden?** U-Matrix, hücre ağırlıkları arasındaki mesafeleri ısı haritası gibi gösterir. **Yüksek mesafe bölgeleri** genellikle **sınır/anomali** alanlarını işaret eder.

```python
u_matrix = som.distance_map()  # (m, n)

plt.figure(figsize=(6,6))
plt.imshow(u_matrix.T, cmap="bone", origin="lower")
plt.title("SOM U-Matrix (Distance Map)")
plt.colorbar()
plt.show()
```

---

# 7) Fraud noktalarını harita üzerinde işaretlemek (hızlı bakış)

```python
# Test verisini BMU'lara (best matching unit) yerleştir
bmu_coords = np.array([som.winner(x) for x in X_test])   # (N_test, 2)

fraud_idx = np.where(y_test==1)[0]
normal_idx = np.where(y_test==0)[0]

plt.figure(figsize=(6,6))
plt.imshow(u_matrix.T, cmap="bone", origin="lower")
plt.scatter(bmu_coords[normal_idx,0], bmu_coords[normal_idx,1], s=3, c="tab:blue", alpha=0.3, label="normal")
plt.scatter(bmu_coords[fraud_idx,0],  bmu_coords[fraud_idx,1],  s=8, c="tab:red",  alpha=0.8, label="fraud")
plt.legend(loc="upper right")
plt.title("SOM Haritası: Test noktaları (Kırmızı=Fraud)")
plt.show()
```

> Fraud’ların **yüksek U-Matrix bölgelerine** veya belirli hücre adalarına daha çok düşmesi beklenir.

---

# 8) SOM tabanlı **anomaly score** ve değerlendirme

**Fikir:** Her örnek için **BMU’ya uzaklık** (quantization error) bir “anomali skoru”dur. **Ne kadar uzak → o kadar anormal.**

```python
# Ağırlık matrisinden BMU vektörünü alıp örnekle arasındaki mesafe
def som_quantization_error(som, x):
    w = som.get_weights()  # (m, n, input_len)
    bmu = som.winner(x)
    w_bmu = w[bmu]         # (input_len,)
    return np.linalg.norm(x - w_bmu)

# Test set skorları
scores_test = np.array([som_quantization_error(som, x) for x in X_test])

# ROC-AUC ve PR-AUC (etikete göre)
roc = roc_auc_score(y_test, scores_test)
ap  = average_precision_score(y_test, scores_test)
print(f"SOM Anomaly Score -> ROC-AUC: {roc:.4f} | PR-AUC: {ap:.4f}")

# PR eğrisi / ROC eğrisi
prec, rec, thr = precision_recall_curve(y_test, scores_test)
fpr, tpr, _ = roc_curve(y_test, scores_test)

plt.figure(figsize=(11,4))
plt.subplot(1,2,1)
plt.plot(fpr, tpr); plt.plot([0,1],[0,1],'k--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC"); plt.grid(alpha=0.3)
plt.subplot(1,2,2)
plt.plot(rec, prec)
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall"); plt.grid(alpha=0.3)
plt.tight_layout(); plt.show()
```

> **Not:** Fraud oranı **çok düşük** olduğu için **PR-AUC** özellikle anlamlıdır. ROC-AUC yüksek görünse bile PR-AUC düşük olabilir; bu normal.

---

# 9) Basit eşik seçimi (en anormal %k yakala)

**Senaryo:** “En riskli %1”i alarm verelim gibi. Aşağıda **üst yüzdelik** ile eşik seçiyoruz.

```python
def topk_threshold(scores, top_ratio=0.01):
    k = max(1, int(len(scores)*top_ratio))
    return np.sort(scores)[-k]

th = topk_threshold(scores_test, top_ratio=0.01)  # en anormal %1
y_pred = (scores_test >= th).astype(int)

print(classification_report(y_test, y_pred, digits=4))
```

> Bu yaklaşım **operasyonel** bir bakış: kuruma “günde en riskli şu kadar işlemi incele” dersen, **top-k** alarm pratik olur.

---

# 10) K-Means + Elbow (karşılaştırma amaçlı)

**Neden?** Gözetimsiz bir referans yöntemle kıyas. K-Means kümelerinde **fraud oranlarına** da bakacağız.

```python
# Elbow grafiği (inertia)
inertias, ks = [], range(2, 13)
for k in ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X_train)
    inertias.append(km.inertia_)

plt.plot(ks, inertias, marker="o")
plt.title("K-Means Elbow (Inertia)")
plt.xlabel("k"); plt.ylabel("inertia"); plt.grid(alpha=0.3)
plt.show()

# Örnek bir k ile test kümeleri ve fraud oranları
k = 8
km = KMeans(n_clusters=k, n_init=10, random_state=42).fit(X_train)
test_labels = km.predict(X_test)

# Her kümede fraud oranı
fraud_rates = []
for i in range(k):
    idx = (test_labels == i)
    if idx.sum() > 0:
        fraud_rate = y_test[idx].mean()
    else:
        fraud_rate = np.nan
    fraud_rates.append(fraud_rate)

pd.DataFrame({"cluster": range(k), "fraud_rate": fraud_rates}).sort_values("fraud_rate", ascending=False)
```

> Burada amaç, bazı kümelerin **fraud-yoğun** olduğunu görmek. SOM haritasında gördüğün **anomali adaları**, K-Means tarafında **yüksek fraud oranlı** 1–2 kümeye karşılık gelebilir.

---

## Mini rehber (ne oldu/niye oldu?)

* **StandardScaler**: Tüm öznitelikleri benzer ölçeğe getirir → mesafe tabanlı (SOM/KMeans) yöntemler sağlıklı çalışır.
* **PCA (opsiyonel)**: Gürültüyü azaltır, hız kazandırır; SOM’un stabilitesini artırır.
* **SOM**: 2D grid üzerinde benzer örnekleri komşu hücrelere yerleştirir.

  * **U-Matrix**: Hücre ağırlıkları arası mesafe ısı haritası (sınırlar/parçalanmalar = anomali sinyali).
  * **Quantization error**: Örneğin BMU’suna uzaklığı = **anomali skoru**.
* **ROC/PR**: SOM skoru ile gerçek etiketleri kıyaslar; PR özellikle dengesiz veri için anlamlıdır.
* **Top-k eşik**: İşe alıştırması kolay bir alarm kuralı: “en anormal %x”.
* **K-Means + Elbow**: Gözetimsiz referans; fraud oranlarıyla kümeleri yorumlayabilirsin.

---

## Varyasyonlar (denemek istersen)

* **SOM grid boyutu**: 15×15 / 25×25 dene; `sigma` ve iteration’ları biraz oyna.
* **Anomaly skoru**: BMU uzaklığı yerine **BMU komşu ort. U-Matrix değeri** (veya BMU ağırlığı ile “local density”) de skora dönüştürülebilir.
* **Sadece `Amount` ve `Time`’ı özel işle**: Log(Amount+1), zaman pencereleri vb. özellik mühendisliği ekleyebilirsin.
* **Karşılaştırma algoritması**: Isolation Forest / LOF ekle (skorları PR-AUC ile kıyasla).

---

Hazırsan bu notebook’la çalış; grafik/sonuçlardan sonra “skoru nasıl iyileştiririz?” diye birlikte ince ayar yaparız (SOM hiperparametreleri, PCA oranı, top-k, vs.).


Güzel çalışma! Çıktıları birlikte okuyalım ve hızlı iyileştirmelerle “daha anlamlı” sonuçlar alalım.

## Ne görüyoruz?

* **ROC-AUC = 0.59, PR-AUC = 0.548** → ayırma gücü zayıf (pozitif oranı ≈0.44, yani PR-AUC tabanı ≈0.44; biraz üstü).
* **Classification report**’ta `recall(1)=0.016` → neredeyse hiç fraud yakalanmamış. Bunun nedeni büyük olasılıkla **top-k eşiği**ni **%1** seçmen: 138 testten 1–2 örneği “anomali” saydı → **çok düşük recall**.
* **K-Means** tabloda bazı kümelerin **çok yüksek fraud oranı** var (örn. cluster 0: 1.00). Bu, veri uzayında **fraud yoğun adacıklar** olduğunu gösteriyor – faydalanabiliriz.

## Hızlı teşhis (nerede tıkanıyoruz?)

1. **Feature set**: Paylaştığın tabloda **`CustomerID`** var. Bu **kimlik/anahtar** niteliği taşıyor; davranış bilgisi içermiyor. **Mutlaka çıkar.**
2. **Boyut/ölçek**: 15 → 14 PCA kalmış; zaten düşük boyut. PCA’yı kapatıp sadece **StandardScaler** ile denemek çoğu kez daha iyi oluyor.
3. **SOM ayarları**: `20×20` (400 hücre) **552 train** için **sparse** kalabilir. Daha kompakt bir harita genelde daha iyi ayrışıyor.

---

## 1. Kolay kazançlar: eşiği düzelt + kimlik sütununu at

Varsayılan **top-k=1%** yerine, **PR eğrisinden** eşik seçelim (ör. **F1’i maksimize eden** ya da **Precision≥0.8** koşullu en yüksek recall).

```python
from sklearn.metrics import f1_score, precision_recall_curve

# 1) CustomerID'yi düş
X = df.drop(columns=["Class", "CustomerID"])  # <— ÖNEMLİ
y = df["Class"].values.astype(int)

# 2) Sadece StandardScaler kullan (PCA'yı kapatıp tekrar dene)
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# 3) (Önceki SOM'u bu X_train ile yeniden eğit)
# ... som = MiniSom(...); som.train_random(X_train, ...)

# 4) QE skorları (test)
scores_test = np.array([som_quantization_error(som, x) for x in X_test])

# 5) F1-optimal eşik
prec, rec, thr = precision_recall_curve(y_test, scores_test)
f1s = 2*prec[:-1]*rec[:-1] / (prec[:-1]+rec[:-1] + 1e-12)
t_star = thr[f1s.argmax()]
y_pred = (scores_test >= t_star).astype(int)

from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
print("ROC-AUC:", roc_auc_score(y_test, scores_test))
print("PR-AUC :", average_precision_score(y_test, scores_test))
print(classification_report(y_test, y_pred, digits=4))
```

> Beklenti: **recall(1)** keskin biçimde artar, F1 toparlar. (Precision biraz düşebilir; bu bir **iş tercihi**.)

İş ihtiyacına göre **Precision≥0.8** gibi bir hedef de koyabilirsin:

```python
target_p = 0.80
idx = np.where(prec[:-1] >= target_p)[0]
i = idx[np.argmax(rec[idx])] if len(idx) else f1s.argmax()
t_star = thr[i]
y_pred = (scores_test >= t_star).astype(int)
```

---

## 2. SOM hiperparametreleri: haritayı “sıkılaştır”

552 train örneği için şu kılavuz iyi işliyor:

* **Harita boyutu:** `~12×12` veya `~15×15` (400 → 144/225’e düşür)
* **Sigma:** 1.5–2.0
* **Learning rate:** 0.5 → iterasyonla azaltma (MiniSom kendi azaltır)
* **Iterasyon:** `~10–20 × N_train` (5× ile hızlı test, iyi durursa artır)

```python
m, n = 12, 12
som = MiniSom(x=m, y=n, input_len=X_train.shape[1], sigma=1.8, learning_rate=0.5,
              neighborhood_function='gaussian', random_seed=42)
som.random_weights_init(X_train)
som.train_random(X_train, 15*X_train.shape[0], verbose=True)
```

> Daha kompakt haritalar genelde **U-Matrix’te ayrımı** netleştirir, **QE skorunun** sinyalini güçlendirir.

---

## 3. (Yarı denetimli) **BMU Fraud Yoğunluğu** skoru

SOM unsupervised; ama **etiketleri sadece “skor fonksiyonu” öğrenmek için kullanabilirsin**:
Her BMU hücresi için **train set fraud oranı**nı hesapla; testte o hücreye düşen örneğe bu oranı **anomaly/fraud skoru** olarak ver.

```python
# Train noktalarını hücrelere ata
bmu_train = np.array([som.winner(x) for x in X_train])

# Her hücrenin fraud oranını hesapla
from collections import defaultdict
cell_counts = defaultdict(int)
cell_fraud  = defaultdict(int)
for (cx, cy), lab in zip(bmu_train, y_train):
    cell_counts[(cx, cy)] += 1
    cell_fraud[(cx, cy)]  += int(lab)

cell_rate = {k: (cell_fraud[k] / cell_counts[k]) for k in cell_counts.keys()}

# Test için “fraud propensity score”
bmu_test = np.array([som.winner(x) for x in X_test])
propensity = np.array([cell_rate.get(tuple(c), 0.0) for c in bmu_test])  # görülmeyen hücreler 0

print("BMU-propensity ROC-AUC:", roc_auc_score(y_test, propensity))
print("BMU-propensity PR-AUC :", average_precision_score(y_test, propensity))
```

> Bu yaklaşım **K-Means küme fraud oranı** fikrine benzer ama **topolojik (SOM)** bir ağ üzerinde çalışır. Çoğu zaman **QE skorundan daha güçlü** olur.

İstersen **iki skoru birleştir** (ör. `final_score = α*QE + (1-α)*propensity` ya da bir **logit regresyon** ile):

```python
from sklearn.linear_model import LogisticRegression
S = np.vstack([scores_test, propensity]).T
clf = LogisticRegression().fit(S, y_test)  # (gerçekte train/valid ayrımı yap)
final = clf.predict_proba(S)[:,1]
print("Combo ROC-AUC:", roc_auc_score(y_test, final))
print("Combo PR-AUC :", average_precision_score(y_test, final))
```

---

## 4. K-Means ile karşılaştırmayı güçlendir

K-Means tarafında yaptığın gibi **kümelerin fraud oranını** haritala ve bunu da **skor** olarak kullan:

```python
km = KMeans(n_clusters=8, n_init=10, random_state=42).fit(X_train)
train_k = km.predict(X_train)
test_k  = km.predict(X_test)

# Küme fraud oranları (train)
k_rates = {i: (y_train[train_k==i].mean() if (train_k==i).any() else 0.0)
           for i in range(8)}

k_score = np.array([k_rates[c] for c in test_k])
print("KMeans-propensity ROC-AUC:", roc_auc_score(y_test, k_score))
print("KMeans-propensity PR-AUC :", average_precision_score(y_test, k_score))
```

> Böylece **SOM-propensity** ile **K-Means-propensity**’yi kıyaslayıp, istersen **ensemble** (basit ortalama) yaparsın.

---

## 5. Son mini-checklist

* [x] **`CustomerID`** gibi kimlik sütunlarını **at**
* [x] PCA’yı **kapatıp** bir dene (sadece `StandardScaler`)
* [x] SOM haritasını **12×12 / 15×15** yap, `sigma≈1.8`
* [x] **Eşik seçimini** PR eğrisinden yap (**F1-opt** ya da **Precision≥x**)
* [x] (İsteğe bağlı) **BMU fraud yoğunluğu** skoru + QE ile **birleştir**
* [x] K-Means propensity ile **karşılaştır**; gerekiyorsa **ensemble**

Bu düzeltmelerle PR-AUC ve (seçtiğin eşik stratejisine göre) F1/Recall belirgin şekilde artacaktır. Dilersen çıktını (yenilenen ROC/PR, report) at; birlikte ince ayarı (harita boyutu/σ/iterasyon) optimize edelim.
